# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [84]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [85]:
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [86]:
# Selección de características clave para el modelo
caracteristicas_clave = [
    "HomePlanet", "CryoSleep", "Cabin", "Destination", "Age", "VIP",
    "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"
]

In [87]:
# Reduzco el dataset a las características seleccionadas
espacio_procesado = spaceship[caracteristicas_clave + ["Transported"]].copy()

In [88]:
# Simplificos los valores categóricos complejos
espacio_procesado["Cabin"] = espacio_procesado["Cabin"].str[0]

In [89]:
# Manejo de valores faltantes
imputador = SimpleImputer(strategy="most_frequent")
espacio_procesado = pd.DataFrame(imputador.fit_transform(espacio_procesado), columns=espacio_procesado.columns)

In [90]:
# Codificación de variables categóricas
espacio_procesado = pd.get_dummies(espacio_procesado, drop_first=True)

In [91]:
# Asegurar que todas las columnas numéricas sean correctamente identificadas y procesadas
columnas_numericas = espacio_procesado.select_dtypes(include=["number"]).columns

In [92]:
# Verificar las columnas antes de aplicar el escalado
columnas_numericas

Index([], dtype='object')

In [93]:
# Identificar las columnas categóricas que necesitan codificación
columnas_categoricas = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"]

In [94]:
# Procesar solo estas columnas categóricas con get_dummies
espacio_procesado = spaceship[columnas_categoricas + ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Transported"]].copy()


In [95]:
# Simplificar la columna Cabin a solo "deck"
espacio_procesado["Cabin"] = espacio_procesado["Cabin"].str[0]

In [96]:
# Manejo de valores faltantes
imputador = SimpleImputer(strategy="most_frequent")
espacio_procesado = pd.DataFrame(imputador.fit_transform(espacio_procesado), columns=espacio_procesado.columns)


In [97]:
# Dummies de variables categóricas
espacio_procesado = pd.get_dummies(espacio_procesado, columns=columnas_categoricas, drop_first=True)


In [98]:
# Escalo las características numéricas
columnas_numericas = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
escalador = StandardScaler()
espacio_procesado[columnas_numericas] = escalador.fit_transform(espacio_procesado[columnas_numericas])


**Perform Train Test Split**

In [99]:
# Dividir en características (X) y etiqueta (y)
X = espacio_procesado.drop("Transported", axis=1)
y = espacio_procesado["Transported"]

In [100]:
# Dividir los datos
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.3, random_state=42)


In [101]:
# Confirmo las formas de los conjuntos divididos
X_entrenamiento.shape, X_prueba.shape, y_entrenamiento.shape, y_prueba.shape

((6085, 19), (2608, 19), (6085,), (2608,))

In [102]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [103]:
# Importar BaggingClassifier para Bagging y Pasting
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [104]:
# Configuro modelos para Bagging y Pasting
modelos_ensemble = {
    "Bagging": BaggingClassifier(
        estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True, random_state=42
    ),
    "Pasting": BaggingClassifier(
        estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=False, random_state=42
    ),
}

In [105]:
# Convierto la variable objetivo a enteros
y_entrenamiento = y_entrenamiento.astype(int)
y_prueba = y_prueba.astype(int)

# Entreno y evaluo los modelos
resultados_ensemble = {}

for nombre, modelo in modelos_ensemble.items():
    # Entreno el modelo
    modelo.fit(X_entrenamiento, y_entrenamiento)
    # Realizo predicciones en el conjunto de prueba
    y_pred = modelo.predict(X_prueba)
    # Calculo la precisión
    precision = accuracy_score(y_prueba, y_pred)
    # Guardo resultados
    resultados_ensemble[nombre] = precision

In [106]:
resultados_ensemble

{'Bagging': 0.7818251533742331, 'Pasting': 0.7342791411042945}

- Random Forests

In [ ]:
# Entreno y evaluo un modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_entrenamiento, y_entrenamiento)

RandomForestClassifier(random_state=42)

In [108]:
# Realizo predicciones en el conjunto de prueba
y_pred_rf = random_forest.predict(X_prueba)

In [109]:
# Calculo la precisión del modelo
precision_rf = accuracy_score(y_prueba, y_pred_rf)

In [119]:
precision_rf

0.7841257668711656

- Gradient Boosting

In [ ]:
# Entreno y evaluo un modelo de Gradient Boosting
gradient_boosting = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gradient_boosting.fit(X_entrenamiento, y_entrenamiento)

GradientBoostingClassifier(random_state=42)

In [112]:
# Realizo predicciones en el conjunto de prueba
y_pred_gb = gradient_boosting.predict(X_prueba)

In [113]:
# Calculo la precisión del modelo
precision_gb = accuracy_score(y_prueba, y_pred_gb)

In [114]:
precision_gb

0.7933282208588958

- Adaptive Boosting

In [ ]:
# Entreno y evaluo un modelo de AdaBoost
adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
adaboost.fit(X_entrenamiento, y_entrenamiento)

AdaBoostClassifier(learning_rate=0.1, n_estimators=100, random_state=42)

In [116]:
# Realizo predicciones en el conjunto de prueba
y_pred_ab = adaboost.predict(X_prueba)

In [117]:
# Calculo la precisión del modelo
precision_ab = accuracy_score(y_prueba, y_pred_ab)

In [118]:
precision_ab

0.7388803680981595

Which model is the best and why?

¿Por qué Gradient Boosting es el mejor?
Mayor Precisión:
    Gradient Boosting alcanzó una precisión de 79.33%, superando a Random Forest (78.41%), Bagging (78.18%) y AdaBoost (73.88%).
Ventajas de Gradient Boosting:
    Este método construye los árboles de manera secuencial, corrigiendo los errores cometidos por los árboles anteriores. Esto le permite enfocarse en las instancias más difíciles de predecir, mejorando el rendimiento global.
Menor Riesgo de Overfitting:
    Comparado con Random Forest y Bagging, que se basan en el muestreo aleatorio (bootstrapping), Gradient Boosting tiende a ser más efectivo en datos estructurados como este, cuando se ajusta correctamente.
Control sobre el Aprendizaje:
    Los hiperparámetros como la tasa de aprendizaje (learning_rate) y el número de estimadores (n_estimators) permiten un control más detallado del modelo, evitando el sobreajuste y maximizando su rendimiento.